In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import importlib
import time

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy import units as u
import h5py

import utils as ut

utils_fold = "C:/Users/Pere/Documents/code/utils/"
sys.path.append(utils_fold)
import mutils as mut
from mutils.plot_utils import Figure
from mutils.plot_utils import Figure3D
importlib.reload(mut)

<module 'mutils' from 'C:\\Users/Pere/Documents/code/utils\\mutils\\__init__.py'>

In [2]:
simulation_name = 'galaxy_mergers'
output_folder = 'output_2'

snapshot_paths = ut.get_snapshots(f'../data/{simulation_name}/{output_folder}/')
M = len(snapshot_paths)

file_first = h5py.File(snapshot_paths[0], 'r')
file_last = h5py.File(snapshot_paths[-1], 'r')

print('Time Start:', file_first['Header'].attrs['Time'], 's')
print('Time End:', file_last['Header'].attrs['Time'], 'g4-time')
print('Time End:', file_last['Header'].attrs['Time']*ut.fact_GU_to_Gyr, 'Gyr')

print('Redshift Start:', file_first['Header'].attrs['Redshift'])
print('Redshift End:', file_last['Header'].attrs['Redshift'])

N1 = np.array(file_first['PartType1']['Coordinates']).shape[0]
N2 = np.array(file_first['PartType2']['Coordinates']).shape[0]

print('Number of Particles Type 1:', N1)
print('Number of Particles Type 2:', N2)

x_min, y_min, z_min = np.min(file_first['PartType1']['Coordinates'], axis=0)
x_max, y_max, z_max = np.max(file_first['PartType1']['Coordinates'], axis=0)

print('x0_min, y0_min z0_min', x_min, y_min, z_min)
print('x0_max, y0_max z0_max', x_max, y_max, z_max)

times = [file['Header'].attrs['Time'] for file in [h5py.File(snapshot_path, 'r') for snapshot_path in snapshot_paths]]

PM0, PM1, PM2 = file_first['Header'].attrs['MassTable']

PM1 = PM1*ut.M_gadget.to(u.Msun).value
PM2 = PM2*ut.M_gadget.to(u.Msun).value

print(f'Mass of Particles Type 1:  {PM1:0.2e} Msun')
print(f'Mass of Particles Type 2:  {PM2:0.2e} Msun')

BMG1 = N1 * PM1 / 2
BMG2 = N2 * PM2 / 2

print(f'Baronic Mass of Single Galaxy:  {BMG1:0.3e} Msun')
print(f'Dark Mass of Single Galaxy:  {BMG2:0.3e} Msun')
print(f'Total Mass of Single Galaxy:  {BMG1+BMG2:0.3e} Msun')
print(f'Total Mass Simulation:  {2*BMG1+2*BMG2:0.3e} Msun')


Time Start: 0.0 s
Time End: 10.0 g4-time
Time End: 9.7779235 Gyr
Redshift Start: 0.0
Redshift End: 0.0
Number of Particles Type 1: 40000
Number of Particles Type 2: 20000
x0_min, y0_min z0_min -191.4137 -133.08165 -99.40706
x0_max, y0_max z0_max 192.29349 131.80861 99.05411
Mass of Particles Type 1:  1.05e+07 Msun
Mass of Particles Type 2:  2.33e+06 Msun
Baronic Mass of Single Galaxy:  2.093e+11 Msun
Dark Mass of Single Galaxy:  2.326e+10 Msun
Total Mass of Single Galaxy:  2.326e+11 Msun
Total Mass Simulation:  4.652e+11 Msun


In [4]:
steps = [0, 16, 100]

subplots = (len(steps), 3)
fig_size = 1920
ratio = 3 / len(steps) * 0.95

ts = 1.5
Fig = Figure(fig_size=fig_size, subplots=subplots, ratio=ratio, theme='default', wspace=0.025, hspace=0.05, ts=ts)
fs = Fig.fs
axes = Fig.axes
axes_flat = Fig.axes_flat


lw = 0.1

for j, step in enumerate(steps):

    

    file = h5py.File(snapshot_paths[step], 'r')
    pos_1 = np.array(file['PartType1']['Coordinates'])
    pos_2 = np.array(file['PartType2']['Coordinates'])

    resolution = [100]*3
    lim = 199

    dh = 2*lim/resolution[0]

    ranges = [(-lim, lim), (-lim, lim), (-lim, lim)]

    # make histogram 2d of XY plane
    hist_1, edges = np.histogramdd(pos_1[:, :], bins=resolution, range=ranges)
    hist_2, _ = np.histogramdd(pos_2[:, :], bins=resolution, range=ranges)

    hist_1 = hist_1 * PM1 / dh**2
    hist_2 = hist_2 * PM2 / dh**2

    hist = hist_1 + hist_2

    # mask values in hist below 1
    # hist_1[hist_1 < 1] = np.nan
    # hist_2[hist_2 < 1] = np.nan
    # hist[hist < 1] = np.nan


    

    axes_row = [axes[j][l] for l in range(3)]
    

    text_lab = ['XY', 'XZ', 'YZ']

    for ii, i in enumerate([2, 1, 0]):
        ax = axes[ii][j]

        projection = np.log(np.sum(hist, axis=i))
        projection_1 = np.log(np.sum(hist_1, axis=i))
        projection_2 = np.log(np.sum(hist_2, axis=i))

        ax.imshow((projection.T)[::-1,:], cmap='binary_r', 
                  extent=[-lim, lim, -lim, lim],
                  vmin=11.5, vmax=20
                  )
        #ax.pcolormesh(edges[0], edges[1], (projection.T)[::-1,:], cmap='binary_r')

        if j == 2:
            # add colorbar to ax
            cax = ax.inset_axes([1.05, 0.01, 0.075, 0.975])
            cax.tick_params(labelsize=ts*fs,
                            size=0.5*fs, width=0.15*fs,
                            pad=0.1*fs)

            cbar = plt.colorbar(cax=cax, mappable=ax.images[0], orientation='vertical')
            cbar.set_label(r'$\log \sigma$ [M$_\odot$ kpc$^{-2}$]', fontsize=fs*ts)

            for spine in cax.spines.values():
                spine.set_linewidth(fs*0.15)

            # ax.text(0.95, 0.05, , transform=ax.transAxes, fontsize=fs*ts, color='white', ha='right')


        colors = ['purple', 'c']
        for k, proj in enumerate([projection_1, projection_2]):
            contours = ax.contour(edges[0][:-1], edges[1][:-1], 
                                proj.T, 
                            levels=[15], 
                            colors=colors[k], zorder=2,
                            linewidths=lw*fs)
            #ax.clabel(contours, inline=1, fontsize=12, fmt='%1.1f')


        ax.text(0.05, 0.9, text_lab[ii], transform=ax.transAxes, fontsize=fs*ts, color='white')
        time_val = times[step]*ut.fact_GU_to_Gyr
        
        if ii != 2:
            # set xticks label size equal to 2
            ax.tick_params(axis='x', labelcolor='w')

        if ii == 0:
            ax.set_title(f'$t={time_val:0.2f}$ Gyr', fontsize=fs*ts,
                pad=2*fs)

        if ii == 2:
            ax.set_xlabel('x [kpc]', fontsize=fs*ts)

        if j == 0:
            ax.set_ylabel('y [kpc]', fontsize=fs*ts)
    #     # set title
    #     ax.set_title(titles[j], fontsize=fs*ts,
    #                     pad=2*fs)

    # if i == 1:
    #     ax.set_xlabel('x [kpc]', fontsize=fs*ts)

    # if j == 0:
    #     ax.set_ylabel('y [kpc]', fontsize=fs*ts)

    for ax in axes_flat:
        ax.set_facecolor('black')
        ax.grid(color='white',alpha=0.2)
        # ax.set_xticks([])
        # ax.set_yticks([])

savepath = f'../figures/{simulation_name}/figure_1.jpg'
Fig.save(savepath, bbox_inches='tight')
plt.close()

C:\Users\Pere\AppData\Local\Temp\ipykernel_28028\3017318686.py:56: RuntimeWarning: divide by zero encountered in log
  projection = np.log(np.sum(hist, axis=i))
C:\Users\Pere\AppData\Local\Temp\ipykernel_28028\3017318686.py:57: RuntimeWarning: divide by zero encountered in log
  projection_1 = np.log(np.sum(hist_1, axis=i))
C:\Users\Pere\AppData\Local\Temp\ipykernel_28028\3017318686.py:58: RuntimeWarning: divide by zero encountered in log
  projection_2 = np.log(np.sum(hist_2, axis=i))
